# Look at the AUG 1 flux tube UQ results

Applying various fitting schemes to the AUG 1 flux tube GEM data

- GPR
- various linear multivariate options
- various SVM options
- various degree polynomial multivariate options 

## A pretty standard set of imports

In [ ]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared, Matern, ConstantKernel
import pickle, gzip
import itertools
from timeit import default_timer as timer
import resource, sys
import functools
import operator

## Read the data

In [ ]:
G = pd.read_csv('../../data/gem_uq_inoutput.csv')
G.describe().T

# Transform the data

In [ ]:
G.te_value = np.log10(G.te_value)
G.ti_value = np.log10(G.ti_value)
G.te_ddrho = np.log10(-G.te_ddrho)
G.ti_ddrho = np.log10(-G.ti_ddrho)
G.te_transp_flux = np.log10(G.te_transp_flux)
G.ti_transp_flux = np.log10(G.ti_transp_flux)
G.describe().T

## Examine correlations

In [ ]:
axes = pd.plotting.scatter_matrix(G, alpha=0.5, diagonal='hist', figsize=(14,8))
corr = np.array(G.corr())
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.show()

## Prepare for fitting

In [ ]:
XLABELS = ['te_value', 'ti_value', 'te_ddrho', 'ti_ddrho']
X = G[XLABELS].values
YLABELS = ['te_transp_flux', 'ti_transp_flux']
Y = G[YLABELS].values

## Fit the data using GPR

In [ ]:
NX = len(XLABELS)
NY = len(YLABELS)
print('Number of points = ', X.shape[0])

kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

start=timer()
GP = []
for F in range(NY): 
    gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
#    %time 
    gp.fit(X, Y[:,F])
    print(YLABELS[F], gp.kernel_)
    GP.append(gp)
end=timer()
print('Calculating GPR took %0.3f seconds' % (end-start))


## Calculate the data at the fit points

In [ ]:
start=timer()
Xt = X.copy()
Yt = np.zeros((Xt.shape[0],NY))
Yts = np.zeros((Xt.shape[0],NY))
for F in range(NY):
  Yt[:,F], Yts[:,F] = GP[F].predict(Xt, return_std=True)
end=timer()
print('Calculating GPR on grid took %0.3f seconds' % (end-start))

RMSE at the points (should be small) and mean of the predicted variance at the points

In [ ]:
print('RMSE =', np.sqrt((Y-Yt)**2).mean(axis=0))

In [ ]:
print('Means of predicted variance =', Yts.mean(axis=0))

## Prepare to plot the data along various cuts

In [ ]:
Xvals = np.array([np.unique(X[:,i]) for i in range(X.shape[1])])

In [ ]:
def vis(slice, axis=None):
    if slice == 0:
        Xt = np.array([i for i in itertools.product(np.linspace(Xvals[0].min(), Xvals[0].max(), 100), 
                                                    [Xvals[1,2]], 
                                                    [Xvals[2,2]],
                                                    [Xvals[3,2]])])
    elif slice == 1:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    np.linspace(Xvals[1].min(), Xvals[1].max(), 100), 
                                                    [Xvals[2,2]],
                                                    [Xvals[3,2]])])
    elif slice == 2:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    [Xvals[1,2]],
                                                    np.linspace(Xvals[2].min(), Xvals[2].max(), 100), 
                                                    [Xvals[3,2]])])
    elif slice == 3:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    [Xvals[1,2]],
                                                    [Xvals[2,2]],
                                                    np.linspace(Xvals[3].min(), Xvals[3].max(), 100))])
    Yt = np.zeros((Xt.shape[0],NY))
    Yts = np.zeros((Xt.shape[0],NY))
    for F in range(NY):
      Yt[:,F], Yts[:,F] = GP[F].predict(Xt, return_std=True)
    
    if axis is None:
        fig, axis = plt.subplots(figsize=(8,6))
    axis.plot(10.0**(Xt[:,slice]), 10.0**(Yt[:,0]), 'r', label=YLABELS[0])
    axis.fill_between(10.0**(Xt[:,slice]), 10.0**(Yt[:,0]-Yts[:,0]), 10.0**(Yt[:,0]+Yts[:,0]), color='red', alpha=0.5)
    axis.plot(10.0**(Xt[:,slice]), 10.0**(Yt[:,1]), 'b', label=YLABELS[1])
    axis.fill_between(10.0**(Xt[:,slice]), 10.0**(Yt[:,1]-Yts[:,1]), 10.0**(Yt[:,1]+Yts[:,1]), color='blue', alpha=0.5)
    axis.set_xlabel(XLABELS[slice])
    #axis.set_legend(loc=0)
    axis.set_title("%s=%0.3f, %s=%0.3f, %s=%0.3f" % 
              functools.reduce(operator.concat, 
                               tuple([(XLABELS[i], 10.0**Xvals[i,2]) for i in range(len(XLABELS)) if i != slice])))

## Plot the data along various cuts

In [ ]:
plt.figure(figsize=(18,8))
for i in range(NX):
    ax=plt.subplot(2,2,i+1)
    vis(i, ax)
plt.subplots_adjust(hspace=0.4)

## Scan over various fitting methods: imports

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import svm
import tabulate

### GPR

In [ ]:
L=[]

Y_pred = np.zeros_like(Y)
for F in range(Y.shape[1]):
    Y_pred[:,F] = GP[F].predict(X, return_std=False)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['GPR', RMSE, VarianceScore])
print(L[-1])

### GPR: effects from differing training fractions

In [ ]:
S={}
for trF in [0.99, 0.98, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.05, 0.02, 0.01]:
    S[str(trF)] = {}
    S[str(trF)]['trF'] = trF
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X, Y, test_size=trF)
    Y_pred = np.zeros_like(Y_tst)
    for F in range(Y.shape[1]): 
        gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
        gp.fit(X_trn, Y_trn[:,F])
        Y_pred[:,F] = gp.predict(X_tst, return_std=False)
    S[str(trF)]['RMSE'] = np.sqrt(mean_squared_error(Y_tst, Y_pred, multioutput='raw_values'))
    S[str(trF)]['VarianceScore'] = r2_score(Y_tst, Y_pred)

#### GPR: effects from differing training fractions --- plot of RMSE

In [ ]:
plt.figure(figsize=(14,7))
plt.plot([S[s]['trF'] for s in np.sort(list(S.keys()))] , [S[s]['RMSE'] for s in np.sort(list(S.keys()))])
plt.xlabel('Test fraction') ; plt.ylabel('RMSE') ; plt.ylim(-0.1,1)
plt.legend(loc=0, labels=YLABELS)
plt.gca().set_prop_cycle(None)
plt.plot([0, 1], np.broadcast_to(RMSE, (2,2)), '-.' )
plt.title('Gaussian process regression in the logarithmic space');

#### GPR: effects from differing training fractions --- plot of VarianceScore

In [ ]:
plt.figure(figsize=(14,7))
plt.plot([S[s]['trF'] for s in np.sort(list(S.keys()))] , [S[s]['VarianceScore'] for s in np.sort(list(S.keys()))])
plt.xlabel('Test fraction') ; plt.ylabel('VarianceScore') ; plt.ylim(0,1.1)
plt.gca().set_prop_cycle(None)
plt.plot([0, 1], [VarianceScore, VarianceScore], '-.' )
plt.title('Gaussian process regression in the logarithmic space');

### Linear regression

In [ ]:
reg = linear_model.LinearRegression(n_jobs=-1)
reg.fit (X, Y)
Y_pred = reg.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['Linear', RMSE, VarianceScore])
print(L[-1])

### Linear regression: effects from differing training fractions

In [ ]:
S={}
for trF in [0.99, 0.98, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.05, 0.02, 0.01]:
    S[str(trF)] = {}
    S[str(trF)]['trF'] = trF
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X, Y, test_size=trF)
    reg = linear_model.LinearRegression(n_jobs=-1)
    reg.fit (X_trn, Y_trn)
    Y_pred = reg.predict(X_tst)
    S[str(trF)]['RMSE'] = np.sqrt(mean_squared_error(Y_tst, Y_pred, multioutput='raw_values'))
    S[str(trF)]['VarianceScore'] = r2_score(Y_tst, Y_pred)

### Linear regression: effects from differing training fractions --- plot of RMSE

In [ ]:
plt.figure(figsize=(14,7))
plt.plot([S[s]['trF'] for s in np.sort(list(S.keys()))] , [S[s]['RMSE'] for s in np.sort(list(S.keys()))] )
plt.xlabel('Test fraction') ; plt.ylabel('RMSE') ; plt.ylim(0,1)
plt.legend(loc=0, labels=YLABELS)
plt.gca().set_prop_cycle(None)
plt.plot([0, 1] , np.broadcast_to(RMSE, (2,2)), '-.' )
plt.title('Linear regression in the logarithmic space');

### Ridge linear regression

In [ ]:
reg = linear_model.Ridge (alpha = 0.5)
reg.fit (X, Y)
Y_pred = reg.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['Ridge', RMSE, VarianceScore])
print(L[-1])

### RidgeCV linear regression

In [ ]:
reg = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])
reg.fit (X, Y)
Y_pred = reg.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['RidgeCV', RMSE, VarianceScore])
print(L[-1])

### Lasso linear regression

In [ ]:
reg = linear_model.Lasso(alpha = 1e-6)
reg.fit (X, Y)
Y_pred = reg.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['Lasso', RMSE, VarianceScore])
print(L[-1])

### Bayesian ridge linear regression

In [ ]:
reg = linear_model.BayesianRidge()
Y_pred = np.zeros_like(Y)
for F in range(Y.shape[1]):  reg.fit (X, Y[:,F]) ; Y_pred[:,F] = reg.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['BayesianRidge', RMSE, VarianceScore])
print(L[-1])

### SVM SVR 

In [ ]:
Y_pred = np.zeros_like(Y)
for F in range(Y.shape[1]): 
    model = Pipeline([('poly', PolynomialFeatures(degree=1, include_bias=False)), ('svm', svm.SVR(kernel='rbf'))])
    RES = model.fit(X, Y[:,F]) ; Y_pred[:,F] = RES.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['SVR', RMSE, VarianceScore])
print(L[-1])

### SVM nuSVR 

In [ ]:
Y_pred = np.zeros_like(Y)
for F in range(Y.shape[1]): 
    model = Pipeline([('poly', PolynomialFeatures(degree=1, include_bias=False)), ('svm', svm.NuSVR(kernel='rbf'))])
    RES = model.fit(X, Y[:,F]) ; Y_pred[:,F] = RES.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['NuSVR', RMSE, VarianceScore])
print(L[-1])

### SVM SVR (C=1, gamma=1)

In [ ]:
Y_pred = np.zeros_like(Y)
for F in range(Y.shape[1]): 
    model = Pipeline([('poly', PolynomialFeatures(degree=1, include_bias=False)), ('svm', svm.SVR(C=1, gamma=1, kernel='rbf'))])
    RES = model.fit(X, Y[:,F]) ; Y_pred[:,F] = RES.predict(X)
print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
RMSE = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
print("RMSE ", RMSE)
# Explained variance score: 1 is perfect prediction
VarianceScore = r2_score(Y, Y_pred)
print('Variance score: %.3f' % VarianceScore)
L.append(['SVR(C=1,gamma=1)', RMSE, VarianceScore])
print(L[-1])

### Multivariate polynomial regression

In [ ]:
S={}
for i in range(13):  #13
    S[i]={}
    model = Pipeline([('poly', PolynomialFeatures(degree=i+1)), ('linear', RidgeCV(alphas=[0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0], fit_intercept=False))])
    RES = model.fit(X, Y)
    S[i]['RES'] = RES
    S[i]['alpha'] = RES.named_steps['linear'].alpha_
    S[i]['coef'] = RES.named_steps['linear'].coef_
    S[i]['intercept'] = RES.named_steps['linear'].intercept_
    Y_pred = RES.predict(X)
    S[i]['degree'] = i+1
    print('Polynomial of degree ', i+1)
    S[i]['MSE'] = mean_squared_error(Y, Y_pred)
    print("Mean squared error: %.3f" % mean_squared_error(Y, Y_pred))
    S[i]['VarianceScore'] = r2_score(Y, Y_pred)
    print('Variance score: %.3f' % r2_score(Y, Y_pred))
    S[i]['RMSE'] = np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values'))
    print("RMSE ", np.sqrt(mean_squared_error(Y, Y_pred, multioutput='raw_values')))

### Multivariate polynomial regression: RMSE vs degree

In [ ]:
plt.figure(figsize=(14,7))
plt.plot([S[s]['degree'] for s in np.sort(list(S.keys()))] , [S[s]['RMSE'] for s in np.sort(list(S.keys()))] )
plt.xlabel('Degree') ; plt.ylabel('RMSE') ; plt.ylim(0,1)
plt.legend(loc=0, labels=YLABELS)
plt.title('RidgeCV regression with polynomial extension\nin the logarithmic space')

### Multivariate polynomial regression: effect of training fraction

In [ ]:
for C in S:
    print('degree ', S[C]['degree'])
    S2={}
    for trF in [0.99, 0.98, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.05, 0.02, 0.01]:
        S2[str(trF)] = {}
        S2[str(trF)]['trF'] = trF
        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X, Y, test_size=trF)
        model = Pipeline([('poly', PolynomialFeatures(degree=S[C]['degree'])), ('linear', RidgeCV(alphas=[0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0], fit_intercept=False))])
        RES = model.fit(X_trn, Y_trn)
        Y_pred = RES.predict(X_tst)
        S2[str(trF)]['RMSE'] = np.sqrt(mean_squared_error(Y_tst, Y_pred, multioutput='raw_values'))
        S2[str(trF)]['VarianceScore'] = r2_score(Y_tst, Y_pred)

    plt.figure(figsize=(14,7))
    plt.plot([S2[s]['trF'] for s in np.sort(list(S2.keys()))] , [S2[s]['RMSE'] for s in np.sort(list(S2.keys()))] )
    plt.xlabel('Test fraction') ; plt.ylabel('RMSE') ; plt.ylim(0,1)
    plt.legend(loc=0, labels=YLABELS)
    plt.gca().set_prop_cycle(None)
    plt.plot([0, 1] , np.broadcast_to(S[C]['RMSE'], (2,2)), '-.' )
    plt.title('RidgeCV regression with polynomial extension %s\nin the normalised logarithmic space' % (S[C]['degree']));

## Summary of results

In [ ]:
LL = []
for l in L:
    LLL=[] ; LLL.append(l[0]) ; LLL.extend(l[1]) ; LLL.append(l[2])
    LL.append(LLL)
for s in S: 
    SSS=[] ; SSS.append('RidgeCV_with_polynomial_extension=%s' % S[s]['degree'])
    SSS.extend(S[s]['RMSE']) ; SSS.append(S[s]['VarianceScore'])
    LL.append(SSS)
headers=['method'] ; headers.extend(YLABELS) ; headers.append('VarianceScore')
print(tabulate.tabulate(LL, tablefmt="simple", headers=headers, floatfmt='5.3f'))